# NYC Rent

**GOAL:** Improve our model for predicting NYC rent prices.

**Objectives**

- Do one-hot encoding of categorical features
- Do univariate feature selection
- Use scikit-learn to fit Ridge Regression models

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

# I. Wrangle Data

In [ ]:
import numpy as np
import pandas as pd

def wrangle(filepath):
  # Import CSV
  df = pd.read_csv(filepath, 
                   parse_dates=['created'],
                   index_col='created')
  
  # Remove outliers
  df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
          (df['price'] <= np.percentile(df['price'], 99.5)) & 
          (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
          (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
          (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
          (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
  
  # Drop high-cardinality categorical variables
  # Austin's proposal
  #df.drop(columns=['description', 'display_address', 'street_address'], inplace=True)

  # Tori's proposal
  drop_cols = [col for col in df.select_dtypes('object')
              if df[col].nunique() > 10]
  df.drop(columns=drop_cols, inplace=True)

  return df

df = wrangle(DATA_PATH+'apartments/renthop-nyc.csv')

In [ ]:
df.head()

,bathrooms,bedrooms,latitude,longitude,price,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
created,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-24 07:54:24,1.5,3,40.7145,-73.9425,3000,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-06-12 12:19:27,1.0,2,40.7947,-73.9667,5465,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-17 03:26:41,1.0,1,40.7388,-74.0018,2850,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-18 02:22:02,1.0,1,40.7539,-73.9677,3275,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2016-04-28 01:32:41,1.0,4,40.8241,-73.9493,3350,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# II. Split Data

Split **target vector** from **feature matrix**.

In [ ]:
target = 'price'
y = df[target]
X = df.drop(columns=target)

Split data into **training** and **test** sets.

In [ ]:
cutoff = '2016-06-01'
mask = X.index < cutoff

X_train, y_train = X.loc[mask], y.loc[mask]
X_test, y_test = X.loc[~mask], y.loc[~mask]

# III. Establish Baseline

**Note:** This is a **regression** problem because we're predictiong the continuous value `'price'`.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = [y_train.mean()] * len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))

Baseline MAE: 1201.8811133682555


# IV. Build `LinearRegression` Model

**NOTES**
  
  - Before we train our model, we need to **encode** our categorical features.
  - For high-dimentional data, your have a few options. First is **dimentionality reduction** using `SelectKBest`.

In [ ]:
# Step 1: Import predictor and transformer classes
from sklearn.linear_model import LinearRegression, Ridge
from category_encoders import OneHotEncoder
from sklearn.feature_selection import SelectKBest

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Step 2: Instantiate transfomers
transformer_1 = OneHotEncoder(use_cat_names=True)
transformer_2 = SelectKBest(k=25) # k is a HYPERPARMETER

# Step 3: Fit transformer 1 to the training data
# NEVER refit your transformer when transforming your TEST data
transformer_1.fit(X_train) 

# Step 4: Do first transformation of training and test data
XT_train = transformer_1.transform(X_train)
XT_test = transformer_1.transform(X_test)

# Step 5: Fit transformer 2 to (transformed) training data
transformer_2.fit(XT_train, y_train)

# Step 6: Do second transformation of training and test data
XTT_train = transformer_2.transform(XT_train)
XTT_test = transformer_2.transform(XT_test)


/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [ ]:
# Step 7a: Instantiate our linear regression predictor
model_lr = LinearRegression()

# Step 8a: Fit model on TRAINING data
model_lr.fit(XTT_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Another option is to use a predictor that does some sort of **regularization**. 

In [ ]:
# Step 7b: Instantiate our ridge regression predictor
model_r = Ridge(alpha=1.0) # alpha is also a HYPERPARAMETER

# Step 8b: Fit model on TRAINING data
model_r.fit(XT_train, y_train) # NOTE: this is the OHE data w/ no SelectKBest

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

# V. Check Metrics

In [ ]:
print('LR training MAE:', mean_absolute_error(y_train, model_lr.predict(XTT_train)))
print('LR test MAE:', mean_absolute_error(y_test, model_lr.predict(XTT_test)))

LR training MAE: 673.3136812759128
LR test MAE: 676.4400492000973


In [ ]:
print('Ridge training MAE:', mean_absolute_error(y_train, model_r.predict(XT_train)))
print('Ridge test MAE:', mean_absolute_error(y_test, model_r.predict(XT_test)))

Ridge training MAE: 672.664072658997
Ridge test MAE: 675.514908956413


# Communicate results

More on this tomorrow...